In [1]:
#import pandas and sqlite
import pandas as pd
import sqlite3 as sl

In [2]:
#connecct to database object
conn = sl.connect('NYT_covid.db')
c = conn.cursor()

In [3]:
#confirm tables present
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print (name[0])

deaths
mask_use
counties
states
us
election


In [10]:
df = pd.read_sql('''SELECT * FROM counties WHERE state = 'Illinois' AND county = 'Cook' ORDER by date asc''',conn)
df

,date,county,state,fips,cases,deaths
0,2020-01-24,Cook,Illinois,17031.0,1,0
1,2020-01-25,Cook,Illinois,17031.0,1,0
2,2020-01-26,Cook,Illinois,17031.0,1,0
3,2020-01-27,Cook,Illinois,17031.0,1,0
4,2020-01-28,Cook,Illinois,17031.0,1,0
...,...,...,...,...,...,...
266,2020-10-16,Cook,Illinois,17031.0,160645,5340
267,2020-10-17,Cook,Illinois,17031.0,162220,5350
268,2020-10-18,Cook,Illinois,17031.0,163912,5357
269,2020-10-19,Cook,Illinois,17031.0,165198,5364


In [11]:
df = pd.read_sql('''SELECT date, county, state, cases as 'Total_cases_to_date'
                    FROM counties 
                    WHERE state = 'Illinois'
                    AND county = 'Cook'
                    ''',conn)
df

,date,county,state,Total_cases_to_date
0,2020-01-24,Cook,Illinois,1
1,2020-01-25,Cook,Illinois,1
2,2020-01-26,Cook,Illinois,1
3,2020-01-27,Cook,Illinois,1
4,2020-01-28,Cook,Illinois,1
...,...,...,...,...
266,2020-10-16,Cook,Illinois,160645
267,2020-10-17,Cook,Illinois,162220
268,2020-10-18,Cook,Illinois,163912
269,2020-10-19,Cook,Illinois,165198


In [15]:
#we are going to explore LAG.  LAG is a window funcgtion that allows you to use values from previous rows.  
#By default, the offset is 1 (meaning the previous row is used)

df = pd.read_sql('''SELECT date, state, county, 
                    cases - LAG (cases,1) OVER (ORDER BY date) as 'new_cases',
                    cases as 'cumulative_cases',
                    deaths - LAG (deaths,1) OVER (ORDER BY date) as 'new_deaths',
                    deaths as 'cumulative_deaths'
                    FROM counties
                    WHERE state = 'Illinois'
                    AND county = 'Cook'
                    ''',conn)
df

,date,state,county,new_cases,cumulative_cases,new_deaths,cumulative_deaths
0,2020-01-24,Illinois,Cook,NaN,1,NaN,0
1,2020-01-25,Illinois,Cook,0.0,1,0.0,0
2,2020-01-26,Illinois,Cook,0.0,1,0.0,0
3,2020-01-27,Illinois,Cook,0.0,1,0.0,0
4,2020-01-28,Illinois,Cook,0.0,1,0.0,0
...,...,...,...,...,...,...,...
266,2020-10-16,Illinois,Cook,1610.0,160645,8.0,5340
267,2020-10-17,Illinois,Cook,1575.0,162220,10.0,5350
268,2020-10-18,Illinois,Cook,1692.0,163912,7.0,5357
269,2020-10-19,Illinois,Cook,1286.0,165198,7.0,5364


In [16]:
# COMBINING WITH PARTITION BY ALLOWS US TO GROUP ALL THE COUNTIES TOGETHER

df = pd.read_sql('''SELECT date, state, county, 
                    cases - LAG (cases,1) OVER (PARTITION BY county ORDER BY date) as 'new_cases',
                    cases as 'cumulative_cases',
                    deaths - LAG (deaths,1) OVER (PARTITION BY county ORDER BY date) as 'new_deaths',
                    deaths as 'cumulative_deaths'
                    FROM counties
                    WHERE state = 'Illinois'
                    ''',conn)
df

,date,state,county,new_cases,cumulative_cases,new_deaths,cumulative_deaths
0,2020-03-20,Illinois,Adams,NaN,1,NaN,0
1,2020-03-21,Illinois,Adams,0.0,1,0.0,0
2,2020-03-22,Illinois,Adams,0.0,1,0.0,0
3,2020-03-23,Illinois,Adams,0.0,1,0.0,0
4,2020-03-24,Illinois,Adams,0.0,1,0.0,0
...,...,...,...,...,...,...,...
20871,2020-10-16,Illinois,Woodford,3.0,549,0.0,11
20872,2020-10-17,Illinois,Woodford,0.0,549,0.0,11
20873,2020-10-18,Illinois,Woodford,0.0,549,0.0,11
20874,2020-10-19,Illinois,Woodford,11.0,560,0.0,11


In [17]:
#using different partitioning gives different results.  I am going to use fips to get this for all states/counties

df = pd.read_sql('''SELECT date, state, county, 
                    cases - LAG (cases,1) OVER (PARTITION BY fips ORDER BY date) as 'new_cases',
                    cases as 'cumulative_cases',
                    deaths - LAG (deaths,1) OVER (PARTITION BY fips ORDER BY date) as 'new_deaths',
                    deaths as 'cumulative_deaths'
                    FROM counties
                    ORDER BY state, county''',conn)
df

,date,state,county,new_cases,cumulative_cases,new_deaths,cumulative_deaths
0,2020-03-24,Alabama,Autauga,NaN,1,NaN,0
1,2020-03-25,Alabama,Autauga,3.0,4,0.0,0
2,2020-03-26,Alabama,Autauga,2.0,6,0.0,0
3,2020-03-27,Alabama,Autauga,0.0,6,0.0,0
4,2020-03-28,Alabama,Autauga,0.0,6,0.0,0
...,...,...,...,...,...,...,...
651163,2020-10-16,Wyoming,Weston,10.0,81,0.0,0
651164,2020-10-17,Wyoming,Weston,7.0,88,0.0,0
651165,2020-10-18,Wyoming,Weston,5.0,93,0.0,0
651166,2020-10-19,Wyoming,Weston,6.0,99,0.0,0


In [24]:
df = pd.read_sql('''SELECT date, state, county, 
                    cases - LAG (cases,1) OVER (PARTITION BY fips ORDER BY date) as 'new_cases',
                    cases as 'cumulative_cases',
                    deaths - LAG (deaths,1) OVER (PARTITION BY fips ORDER BY date) as 'new_deaths',
                    deaths as 'cumulative_deaths'
                    FROM counties
                    ORDER BY state, county''',conn)

df

,date,state,county,new_cases,cumulative_cases,change,new_deaths,cumulative_deaths
0,2020-03-24,Alabama,Autauga,NaN,1,NaN,NaN,0
1,2020-03-25,Alabama,Autauga,3.0,4,4.0,0.0,0
2,2020-03-26,Alabama,Autauga,2.0,6,6.0,0.0,0
3,2020-03-27,Alabama,Autauga,0.0,6,5.0,0.0,0
4,2020-03-28,Alabama,Autauga,0.0,6,5.0,0.0,0
...,...,...,...,...,...,...,...,...
651163,2020-10-16,Wyoming,Weston,10.0,81,81.0,0.0,0
651164,2020-10-17,Wyoming,Weston,7.0,88,88.0,0.0,0
651165,2020-10-18,Wyoming,Weston,5.0,93,93.0,0.0,0
651166,2020-10-19,Wyoming,Weston,6.0,99,99.0,0.0,0


In [37]:
#Now we will add in some rolling averages for new cases and new deaths, we have to use a cte to do this 

df = pd.read_sql('''WITH cte as (SELECT date, state, county, 
                    cases - LAG (cases,1) OVER (PARTITION BY fips ORDER BY date) as 'new_cases',
                    cases as 'cumulative_cases',
                    deaths - LAG (deaths,1) OVER (PARTITION BY fips ORDER BY date) as 'new_deaths',
                    deaths as 'cumulative_deaths'
                    FROM counties
                    ORDER BY state, county)
                    
                    SELECT date, state, county, new_cases, cumulative_cases,
                    AVG(new_cases) OVER (PARTITION BY state, county ORDER BY date ASC rows 6 PRECEDING) as '7-day_avg_new',
                    new_deaths, cumulative_deaths,
                    AVG(new_deaths) OVER (PARTITION BY state, county ORDER BY date ASC rows 6 PRECEDING) as '7-day_avg_death'
                    FROM cte
                    ORDER BY state, county, date
                    ''',conn)
df

,date,state,county,new_cases,cumulative_cases,7-day_avg_new,new_deaths,cumulative_deaths,7-day_avg_death
0,2020-03-24,Alabama,Autauga,NaN,1,NaN,NaN,0,NaN
1,2020-03-25,Alabama,Autauga,3.0,4,3.000000,0.0,0,0.0
2,2020-03-26,Alabama,Autauga,2.0,6,2.500000,0.0,0,0.0
3,2020-03-27,Alabama,Autauga,0.0,6,1.666667,0.0,0,0.0
4,2020-03-28,Alabama,Autauga,0.0,6,1.250000,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...
651163,2020-10-16,Wyoming,Weston,10.0,81,3.857143,0.0,0,0.0
651164,2020-10-17,Wyoming,Weston,7.0,88,4.285714,0.0,0,0.0
651165,2020-10-18,Wyoming,Weston,5.0,93,5.000000,0.0,0,0.0
651166,2020-10-19,Wyoming,Weston,6.0,99,5.857143,0.0,0,0.0


In [38]:
import seaborn as sns